# Computer Vision Lab – Filtering & Edge Detection

This notebook demonstrates **filtering and edge detection** on images.

We will:
- Review image **smoothing (filtering)** as a preprocessing step
- Apply and visualize **Sobel**, **Laplacian**, and **Canny** edge detectors
- Add **noise** to an image (Gaussian and salt-and-pepper)
- Compare the **noise response** of different edge detectors

> **Note:** Place an image file (e.g., `input.jpg`) in the same folder as this notebook or update the `image_path` variable.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Helper Functions

In [ ]:
def show_gray(img, title="Image"):
    """Display a grayscale image using matplotlib."""
    if img is None:
        raise ValueError("Image is None. Check the image path.")
    plt.figure(figsize=(5, 5))
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

def show_side_by_side(img1, img2, title1="Image 1", title2="Image 2"):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img1, cmap='gray')
    plt.title(title1)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(img2, cmap='gray')
    plt.title(title2)
    plt.axis('off')

    plt.show()

## 1. Load Input Image (Grayscale)

We load the input image in **grayscale**, since edge detection is usually applied on single-channel images.

Change the `image_path` below if needed.

In [ ]:
# Change this path if needed
image_path = 'input.jpg'

img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

if img_gray is None:
    raise FileNotFoundError(f"Could not read image from {image_path}. Place an image in this folder or update the path.")

print("Shape (H, W):", img_gray.shape)
print("Data type:", img_gray.dtype)

show_gray(img_gray, "Original Grayscale Image")

## 2. Smoothing / Filtering

Before performing edge detection, it is common to apply some **smoothing** to reduce noise.

We'll try:
- **Box filter (average)**
- **Gaussian filter**
- **Median filter**

Each filter has a different effect on noise and edges.

In [ ]:
# Box filter (average)
blur_box = cv2.blur(img_gray, (5, 5))

# Gaussian filter
blur_gaussian = cv2.GaussianBlur(img_gray, (5, 5), sigmaX=1.0)

# Median filter
blur_median = cv2.medianBlur(img_gray, 5)

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(blur_box, cmap='gray')
plt.title('Box Filter 5x5')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(blur_gaussian, cmap='gray')
plt.title('Gaussian Filter 5x5, σ=1')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(blur_median, cmap='gray')
plt.title('Median Filter 5x5')
plt.axis('off')

plt.show()

## 3. Sobel Edge Detection

The **Sobel operator** approximates the first derivative (gradient) of the image.

It uses two 3×3 kernels:
- One for **horizontal** gradient ($G_x$)
- One for **vertical** gradient ($G_y$)

Edge strength can be approximated by the **magnitude**:

\begin{equation}
G = \sqrt{G_x^2 + G_y^2}
\end{equation}

We will use OpenCV's `cv2.Sobel`.

In [ ]:
# Sobel on original image
Gx = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=3)
Gy = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=3)

# Gradient magnitude
G_mag = np.sqrt(Gx**2 + Gy**2)
G_mag = np.uint8(np.clip(G_mag, 0, 255))

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(np.uint8(np.absolute(Gx)), cmap='gray')
plt.title('Sobel Gx')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(np.uint8(np.absolute(Gy)), cmap='gray')
plt.title('Sobel Gy')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(G_mag, cmap='gray')
plt.title('Sobel Magnitude')
plt.axis('off')

plt.show()

## 4. Laplacian Edge Detection

The **Laplacian** operator computes the second derivative of the image:

\begin{equation}
\nabla^2 f = \frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2}
\end{equation}

It responds strongly to regions of rapid intensity change, but is also **very sensitive to noise**.

We'll use `cv2.Laplacian`.

In [ ]:
lap = cv2.Laplacian(img_gray, cv2.CV_64F, ksize=3)
lap_abs = np.uint8(np.absolute(lap))

show_gray(lap_abs, "Laplacian Edges (ksize=3)")

## 5. Canny Edge Detection

The **Canny edge detector** is a multi-stage algorithm:

1. Smoothing with a Gaussian filter
2. Gradient computation (similar to Sobel)
3. Non-maximum suppression (thin edges)
4. Hysteresis thresholding (strong and weak edges)

In OpenCV: `cv2.Canny(image, threshold1, threshold2)`.

In [ ]:
low_threshold = 100
high_threshold = 200

edges_canny = cv2.Canny(img_gray, low_threshold, high_threshold)

show_gray(edges_canny, f"Canny Edges (low={low_threshold}, high={high_threshold})")

## 6. Adding Noise to the Image

To compare the **noise response** of Sobel, Laplacian, and Canny, we artificially add noise:

- **Gaussian noise** (additive, zero-mean)
- **Salt-and-pepper noise** (random black and white pixels)

Then we will apply edge detectors to:
- Original image
- Noisy image
- Noisy image **after smoothing**

In [ ]:
def add_gaussian_noise(img, mean=0, sigma=20):
    """Add Gaussian noise to a grayscale image."""
    img_float = img.astype(np.float32)
    noise = np.random.normal(mean, sigma, img.shape).astype(np.float32)
    noisy = img_float + noise
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

def add_salt_pepper_noise(img, amount=0.02, s_vs_p=0.5):
    """Add salt-and-pepper noise to a grayscale image."""
    noisy = img.copy()
    num_pixels = img.size
    num_salt = int(amount * num_pixels * s_vs_p)
    num_pepper = int(amount * num_pixels * (1.0 - s_vs_p))

    # Salt (white) noise
    coords = (np.random.randint(0, img.shape[0], num_salt),
              np.random.randint(0, img.shape[1], num_salt))
    noisy[coords] = 255

    # Pepper (black) noise
    coords = (np.random.randint(0, img.shape[0], num_pepper),
              np.random.randint(0, img.shape[1], num_pepper))
    noisy[coords] = 0

    return noisy

noisy_gaussian = add_gaussian_noise(img_gray, sigma=25)
noisy_sp = add_salt_pepper_noise(img_gray, amount=0.03)

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(img_gray, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(noisy_gaussian, cmap='gray')
plt.title('Gaussian Noise')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(noisy_sp, cmap='gray')
plt.title('Salt & Pepper Noise')
plt.axis('off')

plt.show()

## 7. Edge Detection on Noisy Images

We now apply **Sobel**, **Laplacian**, and **Canny** to:

- Original image
- Image with Gaussian noise
- Image with salt-and-pepper noise

and visually compare their responses.

In [ ]:
def sobel_edges(img):
    Gx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    Gy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    G_mag = np.sqrt(Gx**2 + Gy**2)
    G_mag = np.uint8(np.clip(G_mag, 0, 255))
    return G_mag

def laplacian_edges(img):
    lap = cv2.Laplacian(img, cv2.CV_64F, ksize=3)
    lap_abs = np.uint8(np.absolute(lap))
    return lap_abs

def canny_edges(img, low=100, high=200):
    return cv2.Canny(img, low, high)

images = {
    'Original': img_gray,
    'Gaussian Noise': noisy_gaussian,
    'Salt & Pepper Noise': noisy_sp,
}

edge_maps_sobel = {name: sobel_edges(im) for name, im in images.items()}
edge_maps_lap = {name: laplacian_edges(im) for name, im in images.items()}
edge_maps_canny = {name: canny_edges(im) for name, im in images.items()}

# Plot Sobel comparisons
plt.figure(figsize=(12, 4))
for i, (name, emap) in enumerate(edge_maps_sobel.items(), start=1):
    plt.subplot(1, 3, i)
    plt.imshow(emap, cmap='gray')
    plt.title(f'Sobel – {name}')
    plt.axis('off')
plt.tight_layout()
plt.show()

# Plot Laplacian comparisons
plt.figure(figsize=(12, 4))
for i, (name, emap) in enumerate(edge_maps_lap.items(), start=1):
    plt.subplot(1, 3, i)
    plt.imshow(emap, cmap='gray')
    plt.title(f'Laplacian – {name}')
    plt.axis('off')
plt.tight_layout()
plt.show()

# Plot Canny comparisons
plt.figure(figsize=(12, 4))
for i, (name, emap) in enumerate(edge_maps_canny.items(), start=1):
    plt.subplot(1, 3, i)
    plt.imshow(emap, cmap='gray')
    plt.title(f'Canny – {name}')
    plt.axis('off')
plt.tight_layout()
plt.show()

## 8. Quantitative Comparison: Edge Pixel Counts

One simple way to compare noise response is to count the number of **edge pixels** detected.

- More noise → more spurious edge pixels
- Effective noise handling → fewer false edges in noisy images

We count non-zero pixels in the edge maps.

In [ ]:
def count_edge_pixels(edge_img):
    return int(np.count_nonzero(edge_img))

print("Sobel edge pixel counts:")
for name, emap in edge_maps_sobel.items():
    print(f"  {name}: {count_edge_pixels(emap)}")

print("\nLaplacian edge pixel counts:")
for name, emap in edge_maps_lap.items():
    print(f"  {name}: {count_edge_pixels(emap)}")

print("\nCanny edge pixel counts:")
for name, emap in edge_maps_canny.items():
    print(f"  {name}: {count_edge_pixels(emap)}")

## 9. Smoothing Before Edge Detection (Noise Reduction)

Now we apply a **Gaussian filter** to the noisy images **before** edge detection.

This is a common practical approach:
- Reduce noise with smoothing
- Then run an edge detector

We will apply Canny after Gaussian smoothing and compare.

In [ ]:
# Smooth noisy images
gaussian_blur_noisy_gauss = cv2.GaussianBlur(noisy_gaussian, (5, 5), 1.0)
gaussian_blur_noisy_sp = cv2.GaussianBlur(noisy_sp, (5, 5), 1.0)

canny_gauss_smooth = canny_edges(gaussian_blur_noisy_gauss)
canny_sp_smooth = canny_edges(gaussian_blur_noisy_sp)

plt.figure(figsize=(12, 6))

plt.subplot(2, 3, 1)
plt.imshow(noisy_gaussian, cmap='gray')
plt.title('Gaussian Noise')
plt.axis('off')

plt.subplot(2, 3, 2)
plt.imshow(gaussian_blur_noisy_gauss, cmap='gray')
plt.title('Gaussian Noise + Gaussian Blur')
plt.axis('off')

plt.subplot(2, 3, 3)
plt.imshow(canny_gauss_smooth, cmap='gray')
plt.title('Canny after Smoothing (Gaussian Noise)')
plt.axis('off')

plt.subplot(2, 3, 4)
plt.imshow(noisy_sp, cmap='gray')
plt.title('Salt & Pepper Noise')
plt.axis('off')

plt.subplot(2, 3, 5)
plt.imshow(gaussian_blur_noisy_sp, cmap='gray')
plt.title('Salt & Pepper + Gaussian Blur')
plt.axis('off')

plt.subplot(2, 3, 6)
plt.imshow(canny_sp_smooth, cmap='gray')
plt.title('Canny after Smoothing (S&P Noise)')
plt.axis('off')

plt.tight_layout()
plt.show()

print("Canny edge pixel counts after smoothing:")
print("  Gaussian noise + blur:", count_edge_pixels(canny_gauss_smooth))
print("  Salt & pepper + blur:", count_edge_pixels(canny_sp_smooth))

## 10. Summary

In this notebook, you have:

- Reviewed basic **smoothing filters** (box, Gaussian, median)
- Applied three classic **edge detectors**:
  - **Sobel** (first derivative, gradient magnitude)
  - **Laplacian** (second derivative, very sensitive to noise)
  - **Canny** (multi-stage, robust to noise with thresholds and non-max suppression)
- Added **Gaussian noise** and **salt-and-pepper noise** to the image
- Compared noise response of Sobel, Laplacian, and Canny visually and by **edge pixel counts**
- Observed the benefit of **smoothing before edge detection**, especially in noisy conditions

### Suggested Exercises

1. Try different kernel sizes (3×3, 7×7, 9×9) for Gaussian blur and see how edge maps change.
2. Change Canny thresholds and observe effects on noise and edge continuity.
3. Replace Gaussian blur with **median blur** before Canny and compare especially for salt-and-pepper noise.
4. Implement your own simple gradient-based edge detector using convolution with Sobel kernels.
5. Plot histograms of gradient magnitudes and design a simple threshold-based edge detector.

This lab provides a practical foundation for understanding **filtering** and **edge detection** in Computer Vision.